In [1]:
%load_ext google.cloud.bigquery

In [2]:
from google.cloud import bigquery
import datetime
import pandas as pd

In [3]:
client = bigquery.Client()

In [4]:
limits = 'LIMIT 15000'
limits=''
sql_events = f"""
WITH geovisits_h3_indexes AS (
    SELECT *, `carto-os-eu`.h3.LONGLAT_ASH3(longitude, latitude, 10) h3_idx
    FROM `ggo-ppos-bqgis.singlespot.geovisits_paris_matview`
)
select sptId, uuid, latitude, longitude, accuracy, eventId,  
arrival, departure, score, rank, category, feature, placeName, h3_idx 
FROM geovisits_h3_indexes {limits}
"""

In [5]:
rows = client.query(sql_events).result()
df = rows.to_dataframe()
df.head()

,sptId,uuid,latitude,longitude,accuracy,eventId,arrival,departure,score,rank,category,feature,placeName,h3_idx
0,7d21b764-daae-4eb2-8330-a1fa6d0367af,9cf02c39-87ce-4865-9a6d-a46adb199ed3,48.824074,2.373960,48.0,84ea804206129a24f1075da79ffea90da8cc6796e42be0...,2021-07-14 10:34:34 +0200,2021-07-14 10:39:35 +0200,0.346496,1,amenity,veterinary,Obsèques Animales,8a1fb4646327fff
1,8a8f87c1-7dd0-4376-babc-455dd7c3b8b0,7cab399a-d75c-48b1-9c5c-1cb44e4ae337,48.840264,2.341110,42.0,5a5786eed2f59a4119137d0ac227be2f599b7a08ce0224...,2021-07-13 07:41:42 +0200,2021-07-13 09:44:50 +0200,0.512278,1,office,association,Communauté Chrétienne Universitaire,8a1fb4671ccffff
2,c6aa8b4f-857a-42e6-9093-3a6743eaf127,6b8927ca-e0c8-4979-b242-b4e12946a53d,48.880470,2.327291,57.0,938ca94e31e6c22fd016224e2fc845ceab87442c65a1ee...,2021-07-09 11:10:18 +0200,2021-07-09 11:32:37 +0200,0.821422,1,shop,model,Au Pullman,8a1fb4666c37fff
3,1d32bf1b-8694-4598-a569-773c6659303f,ac975d15-555a-4131-8f8d-f90268474e8f,48.844595,2.376132,48.0,f-5f82826cad9aa807265dc87f08a4ab9112faba8b,2021-07-08 17:46:16 +0200,2021-07-08 18:15:50 +0200,0.365783,1,tourism,artwork,None,8a1fb464410ffff
4,b8fbec5b-0de2-4383-9276-de5f22c6fe8d,0b518296-1a3c-4d18-bef8-8f15ed056fe8,48.884469,2.344850,48.0,5a2fa95268e830ecbdbe990a242f467a5c586b1c6c27d0...,2021-07-10 16:02:04 +0200,2021-07-10 17:10:15 +0200,0.565314,1,shop,fabric,Les Coupons de Saint-Pierre,8a1fb4666baffff


In [6]:
df['sptId'] = df['sptId'].astype(str)
df['category'] = df['category'].astype(str)
df['feature'] = df['feature'].astype(str)
df['arrival'] = pd.to_datetime(df['arrival'], format='%Y-%m-%d %H:%M:%S', utc=True)
df['depature'] = pd.to_datetime(df['departure'], format='%Y-%m-%d %H:%M:%S', utc=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 481634 entries, 0 to 481633
Data columns (total 15 columns):
 #   Column     Non-Null Count   Dtype              
---  ------     --------------   -----              
 0   sptId      481634 non-null  object             
 1   uuid       481634 non-null  object             
 2   latitude   481634 non-null  float64            
 3   longitude  481634 non-null  float64            
 4   accuracy   481634 non-null  float64            
 5   eventId    481634 non-null  object             
 6   arrival    481634 non-null  datetime64[ns, UTC]
 7   departure  481634 non-null  object             
 8   score      481634 non-null  float64            
 9   rank       481634 non-null  int64              
 10  category   481634 non-null  object             
 11  feature    481634 non-null  object             
 12  placeName  325184 non-null  object             
 13  h3_idx     481634 non-null  object             
 14  depature   481634 non-null  datetime

In [ ]:
""" 
h3_g = (df
                          .groupby('h3_idx')
                          .eventId
                          .agg(list)
                          .to_frame("ids")
                          .reset_index())
# Let's count each points inside the hexagon
h3_g['count'] =(h3_g['ids']
                      .apply(lambda ignition_ids:len(ignition_ids))) 
"""

h3_g = df.groupby('h3_idx').h3_idx.agg('count').to_frame('count').reset_index()

In [ ]:
h3_g.head()

In [ ]:
df_w = pd.DataFrame(df[['h3_idx', 'category']])
df_w['count'] = 1
df_w.head()


In [ ]:
h3_pivot = pd.DataFrame(pd.pivot_table(df_w,  columns='category', index='h3_idx', values="count", aggfunc=[len], fill_value=0, margins = True, margins_name='total')).reset_index()
h3_pivot.columns = h3_pivot.columns.map('_'.join).str.replace('len_', '')
h3_pivot = h3_pivot.rename(
    columns={'h3_idx_': 'h3_idx'})
h3_pivot.head()

In [ ]:
# 
indexNames = h3_pivot[h3_pivot['h3_idx'] == 'total'].index
h3_pivot.drop(indexNames, inplace=True)
h3_pivot[h3_pivot['h3_idx'] == 'total']
# h3_pivot.info()


In [ ]:
from shapely.geometry import Polygon
from geojson.feature import *
import h3

In [ ]:
def create_geometry(row, hex_id_field):
  points = h3.h3_to_geo_boundary(row[hex_id_field], True)
  return Polygon(points)

def hexagons_dataframe_to_geojson(df_hex, hex_id_field, value_field,file_output = None):

    list_features = []

    for i, row in df_hex.iterrows():
        feature = Feature(geometry = create_geometry(row, hex_id_field),
                          id = row[hex_id_field],
                          properties = {"value": row[value_field]})
        list_features.append(feature)

    feat_collection = FeatureCollection(list_features)

    if file_output is not None:
        with open(file_output, "w") as f:
            json.dump(feat_collection, f)

    else :
      return feat_collection

In [ ]:
geojson_obj =hexagons_dataframe_to_geojson(h3_pivot, hex_id_field='h3_idx', value_field='total')

In [ ]:
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "notebook_connected"

In [ ]:
fig = (px.choropleth_mapbox(
                    h3_g, 
                    geojson=geojson_obj, 
                    locations='h3_idx', 
                    color='count',
                    color_continuous_scale="Viridis",
                    range_color=(0,h3_g['count'].mean()),                  
                    mapbox_style='carto-positron',
                    zoom=12,
                    center = {"lat": 48.85432452980058, "lon": 2.3459243774414062},
                    opacity=0.7,
                    labels={'count':'# of visits '}))
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
df_names = h3_pivot.columns
df_names = df_names[1:]
df_names

```python
""" 
h3_pivot_corr=h3_pivot[df_names]
correlations= h3_pivot_corr.corr()
fig=plt.figure()
ax=fig.add_subplot(111)
cax=ax.matshow(correlations,vmin=-1,vmax=1)
fig.colorbar(cax)
ticks=np.arange(0,len(df_names),1)
ax.set_xticks(ticks)
ax.set_yticks(ticks)
ax.set_xticklabels(df_names)
ax.set_yticklabels(df_names)
plt.show() """
```

In [ ]:
import seaborn as sns
h3_pivot_corr=h3_pivot[df_names]
correlations= h3_pivot_corr.corr()
mask = np.zeros_like(correlations)  # make mask
mask[np.triu_indices_from(mask)] = True  # mask the upper triangle
fig, ax = plt.subplots(figsize=(11, 9))  # create a figure and a subplot
cmap = sns.diverging_palette(220, 10, as_cmap=True)  # custom color map
"""
sns.heatmap(
    correlations,
    mask=mask,
    cmap=cmap,
    center=0,
    linewidth=0.5,
    cbar_kws={'shrink': 0.5}
)
"""
sns.heatmap(
    correlations,
    annot=True
)